# Module 13: GUIs and Executables
March 26, 2021

Last time we dived deeper into objects and object-oriented programming (OOP) in Python. We saw how to create own classes, discuss the concept of inheritance, and had a quick look at UML class diagrams. Furthermore, we briefly talked about higher-order functions, which exploit that in Python also functions are objects.

Today we take a look at some additional practical topics: Building graphical user interfaces (GUIs) and executables with Python, which can be very useful for making functionality for third parties, especially when they can/must/should not deal with the code directly.

Next time we will discuss how to implement parallel behaviour in Python. 

## Graphical User Interfaces with Tkinter

There is a large number of frameworks and toolkits available for building graphical user interfaces (GUIs) in Python (see e.g. the list at https://wiki.python.org/moin/GuiProgramming). We will focus here on how to create GUIs with Tkinter (see the official reference at https://docs.python.org/3/library/tkinter.html), which is the fairly easy-to-use standard GUI framework included in Python. The following introduction to Tkinter is largely based on the very elaborate “Thinking Tkinter” tutorial available online at http://thinkingtkinter.sourceforge.net/.

The simplest possible Tkinter program is probably the following:


In [ ]:
import tkinter as tk

root = tk.Tk()
root.mainloop()

First the Tkinter library is imported under the name ```tk``` for easier reference. Then an instance of the class ```Tkinter.Tk``` is created, which creates a basic window object. This ```root``` object is the the highest-level GUI component in any Tkinter application, often also referred to as the “toplevel window”. Finally, the ```mainloop``` method of the root object is executed. As the name suggests, it starts the main loop of the application window. This loop runs continuously, waiting for events, handling them when they occur, and only stopping when the window is closed.

![](img/tkinter1.png)

Obviously, what needs to happen from here is to add further components to the root window, and implement the functionality to handle events that occur from interaction of a user with the interface. We cannot cover all possibilities in the scope of this lecture, of course, but the following examples should give you an idea how it works.

Two kinds of GUI components are distinguished in Tkinter: containers and widgets. *Widgets* are all the things that are (usually) visible and do things, such as text fields, drop-down lists, buttons, etc. *Containers* are components that, well, contain other components, especially widgets. The most frequently used container class is ```Frame```.

The following example shows how to add a container and a “Say hello!” button to the empty window from above:


In [4]:
root = tk.Tk()

container = tk.Frame(root)
container.pack() 

hwbutton = tk.Button(container)
hwbutton["text"] = "Say hello!"
hwbutton.pack()

root.mainloop() 

We create a ```root``` object as before. Then we add a frame ```container``` to the base window. This establishes a logical relationship between the ```container``` and ```root```. Furthermore, the ```pack``` method needs to be called to invoke a “geometry manager” and  establish a visual relationship between the object and its parent, to actually make it visible. Similarly, we add a button to the container, set its text and ```pack``` it. Finally, the main loop of the application is started. We get a window with a button that we can click, but nothing else happens, simply because we have not defined yet what should happen (but we will get to that).

![](img/tkinter2.png)

When GUI applications get larger, it is usually advisable to follow the object-oriented programming style rather than the procedure-oriented one, and organize the code in classes. For the example from above, that could look as follows:


In [5]:
class HelloWorldApp:   
    
    def __init__(self,parent):
        self.container = tk.Frame(parent)
        self.container.pack()
        self.hwbutton = tk.Button(self.container)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack()
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()   

Adding additional widgets to the application can be done in the same way as adding the button. The following example shows that instead of configuring widgets by using their dictionaries, this can also be done with the ```configure``` method or directly during their instantiation:

In [6]:
class HelloWorldApp:   
    
    def __init__(self,parent):
        self.container = tk.Frame(parent)
        self.container.pack()
        self.hwbutton = tk.Button(self.container)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack()
        self.hwtext = tk.Label(self.container)
        self.hwtext.configure(text="",background="white")
        self.hwtext.pack()
        self.gbbutton = tk.Button(self.container, text="Goodbye!", \
                                  background="red")
        self.gbbutton.pack()
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()

![](img/tkinter3.png)

Note: If you are on Mac OS X, you might have to run the following code instead to see the button in red:

In [ ]:
# importing tkmacosx
import tkmacosx as tkmac

class HelloWorldApp:   
    
    def __init__(self,parent):
        self.container = tk.Frame(parent)
        self.container.pack()
        self.hwbutton = tk.Button(self.container)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack()
        self.hwtext = tk.Label(self.container)
        self.hwtext.configure(text="",background="white")
        self.hwtext.pack()
        # using Button from tkmacosx
        self.gbbutton = tkmac.Button(self.container, text="Goodbye!", \
                                  background="red")
        self.gbbutton.pack()
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()

The standard way of placing widgets within the container is on top of each other. That is because the default value of the ```side``` parameter of the ```pack``` method is in fact ```top```. By using “bottom”, “left” or “right” alternatively, the orientation can be changed. To avoid unpredictable behavior when e.g. resizing the application window, it is advisable to use the same orientation for all widgets in a container. Change the code above to use ```pack``` always with parameter ```side=”left”``` and see what happens.

If we would like, for example, to place the text field above the two buttons, we can easily do that by using to containers (placed on top of each other), of which one contains the text field and the other the two buttons (next to each other):

In [7]:
class HelloWorldApp:  
    
    def __init__(self,parent):
        self.container1 = tk.Frame(parent)
        self.container1.pack()
        self.hwtext = tk.Label(self.container1)
        self.hwtext.configure(text="",background="white")
        self.hwtext.pack(side="left")
        self.container2 = tk.Frame(parent)
        self.container2.pack()
        self.hwbutton = tk.Button(self.container2)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack(side="left")
        self.gbbutton = tk.Button(self.container2, text="Goodbye!", \
                                  background="red")
        self.gbbutton.pack(side="left")
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()   

![](img/tkinter4.png)

So far for creating a tk application window and adding and arranging GUI elements. Of course we also want the buttons to actually do something when we click on them. To achieve this, we need to do two things: 1) write event handler routines that do the intended things, and 2) bind these routines to the respective widgets and events.

For example, if we want a click on the “Say hello!” button to cause the text “Hello World!” to appear in the text box, and a click on the “Goodbye!” button to cause the window to close, we could define the following two methods in our application class:
```
   def hwbuttonClick(self,event):
       self.hwtext.configure(text="Hello World!")
       
   def gbbuttonClick(self,event):
       self.parent.destroy()
``` 
The first method simply changes the text in the text field, the second one calls the ```destroy``` method of the root object and thus closes the window. Furthermore, we need to register the methods at the respective buttons with the ```bind``` method. The first parameter of ```bind``` is the event that we want to handle (a click with the left mouse button is called ```“<Button-1>”```) and the function that is to be called. See the complete example with event binding below:

In [9]:
class HelloWorldApp:   
    
    def __init__(self,parent):
        self.parent = parent
        
        self.container1 = tk.Frame(parent)
        self.container1.pack()
        self.hwtext = tk.Label(self.container1)
        self.hwtext.configure(text="",background="white")
        self.hwtext.pack(side="left")
        
        self.container2 = tk.Frame(parent)
        self.container2.pack()
        self.hwbutton = tk.Button(self.container2)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack(side="left")
        self.hwbutton.bind("<Button-1>", self.hwbuttonClick)
        self.gbbutton = tk.Button(self.container2, 
                                  text="Goodbye!", background="red")
        self.gbbutton.pack(side="left")
        self.gbbutton.bind("<Button-1>", self.gbbuttonClick)    
        
    def hwbuttonClick(self,event):
        self.hwtext.configure(text="Hello World!") 
        
    def gbbuttonClick(self,event):
        self.parent.destroy()
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()  

![](img/tkinter5.png)

Next to the Frame container, the Canvas container mentioned earlier is often useful to use. Basically, it allows for including all kinds of graphics – self-drawn, generated or imported. Here a small example added to our HelloWorldApp:

In [ ]:
class HelloWorldApp:  
    
    def __init__(self,parent):
        self.parent = parent
        
        self.container0 = tk.Canvas(parent, width=100, height=100)
        self.container0.create_oval(0,0,100,100,fill="yellow")
        self.container0.create_oval(45,45,55,55,fill="red")
        self.container0.create_oval(25,25,35,35,fill="blue")
        self.container0.create_oval(65,25,75,35,fill="blue")
        self.container0.create_arc(25,55,75,80,fill="red",
                                   style="arc",start=180,extent=180)
        self.container0.pack()

        self.container1 = tk.Frame(parent)
        self.container1.pack()
        self.hwtext = tk.Label(self.container1)
        self.hwtext.configure(text="",background="white")
        self.hwtext.pack(side="left")
        
        self.container2 = tk.Frame(parent)
        self.container2.pack()
        self.hwbutton = tk.Button(self.container2)
        self.hwbutton["text"] = "Say hello!"
        self.hwbutton.pack(side="left")
        self.hwbutton.bind("<Button-1>", self.hwbuttonClick)
        self.gbbutton = tk.Button(self.container2, 
                                  text="Goodbye!", background="red")
        self.gbbutton.pack(side="left")
        self.gbbutton.bind("<Button-1>", self.gbbuttonClick)  
        
    def hwbuttonClick(self,event):
        self.hwtext.configure(text="Hello World!")  
        
    def gbbuttonClick(self,event):
        self.parent.destroy()
        
root = tk.Tk()
hwapp = HelloWorldApp(root)
root.mainloop()  

The code now adds another container to the GUI frame, namely a Canvas container on top of the two previously defined containers. We create it with ```width = 100``` and ```height = 100```, meaning that the canvas will have a size of 100x100 pixels. Onto that canvas, we draw a big yellow circle, one red and two blue circles as well as an arc, which together create a nice smiley face. :-) Note that the coordinate system of the canvas starts in the upper left corner, so x,y=0,0 is the coordinate in the upper left, 100,100 the one in the lower left, etc. The drawing functions expect a “bounding box”, i.e. two coordinate pairs that define the rectangle in which the figure is drawn, thus always 4 numbers as parameters in of the methods creating the shapes.

![](img/tkinter6.png)

Finally, note that it also possible to display the name of the app in the window frame, instead of “tk”. All that is required is to add one more line to the main program:
```
root = tk.Tk()
root.title('Hello!')
hwapp = HelloWorldApp(root)
root.mainloop()   
```

So much about the basic principles of creating graphical user interfaces in Python with the Tkinter framework. There are a lot more widgets, events and configuration options to be explored, but they follow the same ideas. More advanced information can also be found in the online documentation and tutorials referenced above.

As mentioned in the beginning, several (other) frameworks and toolkits for building GUIs with Python exist. Using toolkits for GUI design often makes it easier to create more “beautiful” interfaces, but on the other hand the code that they generate automatically can be more difficult to understand. If you are interested in more GUI programming, it might nevertheless be worth investigating them further.

## Creating Executables with PyInstaller

In the scientific community, people often like to share their Python programs as plain source code, or as Jupyter notebooks, so that they can easily make changes to the program to adapt it to their own data analysis problems. Also, open-sourcing code of computational experiments is in line with reproducibility standards, etc. In other areas, for example commercial software development, the situation is often different. Customers should use the developed software, but not see the code. They should not need a development environment, but just be able to run a stand-alone version of the program (for historical reasons usually called “executable”, although that is not a very intuitive term for interpreted languages like Python, which you can in principle always directly execute).

The trick thing with executables (in Python, but also with many other languages) is that they are platform-specific. That is, they can be made for either Windows, Mac OS or Linux platforms, so several versions are needed to make all potential users happy. Furthermore, you can usually only build executables for a specific platform on a machine with the same (kind of) operating system, so professional development teams run different (virtual) machines to be able to do that.

As with many things in the Python ecosystem, there are different frameworks available to “freeze” (generate executables for) Python programs. PyInstaller (https://www.pyinstaller.org/) is one of the few of them that supports all major platforms (e.g. Windows, Mac OS and Linux) and most of the recent Python versions, so it is often a good choice, definitely for a course like this one.

My laptop runs on Linux (Ubuntu 16.04) so we will see how it works there. Generally the process on Windows and Mac OS platforms is the same, but in detail it might differ a bit. The PyInstaller manual at https://pyinstaller.readthedocs.io/en/stable/ provides quite elaborate instructions for all platforms (on the process, but also regarding requirements and common errors), so please refer to that when you try to build executables for your Python programs.

So, let’s assume I have already checked that my system meets the listed requirements. If PyInstaller is not installed already, I can simply do that with the command ```pip install pyinstaller``` in the terminal. Then it’s best to run PyInstaller directly from the directory where the (main) Python program file is located, so I go there first:

![](img/pyinstaller1.png)

In principle, all I have to do now is to call PyInstaller with the (main) .py file of the program that I want to turn into an executable, e.g. ```pyinstaller helloworldapp.py.``` Let’s try:

![](img/pyinstaller2.png) ... ![](img/pyinstaller3.png)

A lot of output, but basically it informs us that it has successfully created the executable. It is in the ```dist/``` directory that it has created in the current directory:

![](img/pyinstaller4.png)

The actual executable file is the one just named `helloworldapp`, executable from the command line by entering `./helloworldapp` (might be slightly different on other platforms, e.g. a `helloworld.exe` file on Windows that can be executed through double-clicking on it). Apparently, there are a lot of other files, too. They are there because the whole required runtime environment (Python interpreter, libraries used, …) need to be put into the executable, too, so that it is really stand-alone. You don’t want your users/customers to deal with a development environment, worry about dependencies, etc. 

With the option ```--onefile``` PyInstaller will pack everything into one single file. That can be easier for users (a single file might look more trustworthy than a large collection of strange-looking files), but in case the program includes related files like a README or License information, they would have to be distributed separately.

![](img/pyinstaller5.png)

Note that Python will automatically detect that modules that are imported by the (main) .py file from which the executable is generated also need to be included in the executable. In case further files are to be included, such as README or other files with additional information about the program, or sample input data files, PyInstaller needs to be told about them (e.g. via the command line). And of course there are many more options and advanced features that can be relevant especially when creating executables for larger, more complex programs, but the PyInstaller website and community should be able the help with that, too.

## Exercises

Please use Quarterfall to submit and check your answers. 

### 1. Number Guessing with GUI (★★★★☆)

In an earlier lecture, we programmed a little command-line number-guessing game. The program would generate a random number between 1 and 10, and then ask the user too guess a number until they hit the right one. When the guess is wrong, it would display a message if it is too large or too small. 

Now create a simple GUI for playing the number guessing game. It should look something like: 

![](img/nggui1.png) ![](img/nggui2.png) ![](img/nggui3.png)

That is, it has a text field (`Label`) to display the different messages. Below this, there are an input field (`Entry`) for the user to enter their guess, a button to check the guess (also triggering the text field to change), and a button to start a new round (generating a new random number and resetting the text and input field).

Optionally, create an executable for this program using pyinstaller.

### 2. QR Code Generator with GUI (★★★★☆)
In a previous exercise you implemented a command-line client for a QR code generation web service. (If you did not complete the exercise, you can use the code from the sample solution as a basis.) Now use the Tkinter framework to build a graphical user interface (GUI) for the functionality. The user should be able to enter a text and the RGB codes for foreground and background color. After a click on a button the QR code is generated and displayed. (The image does not have to be saved as a file, and for simplicity you can always create and process the image in the same format, for example png). 
The GUI should look something like (feel free to make a prettier one):

![](img/qrcode_gui.png)

Hint: For displaying the (png) image obtained from the web service on the canvas, the Tkinter.PhotoImage function and the Tkinter.Canvas.create_image method might be useful.

Optionally, create an executable for this program using pyinstaller.